In [21]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from scipy.stats import skew, kurtosis
from scipy.fft import rfft
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import tqdm

EPS = 1e-9

In [33]:
ferg_shp = 'Fergana_training_samples.shp'
oren_shp = 'Orenburg_training_samples.shp'

g_ferg = gpd.read_file(ferg_shp)
g_oren = gpd.read_file(oren_shp)

print(g_ferg.head())
print(g_oren.shape)

   ID  Cropland                   geometry
0   1         0  POINT (71.74908 41.39632)
1   2         0  POINT (72.90143 41.07219)
2   3         1  POINT (72.46399 40.61906)
3   4         0  POINT (72.19732 41.62759)
4   5         0  POINT (71.31204 41.36946)
(500, 3)


In [ ]:
ferg_shp = 'Fergana_training_samples.shp'
oren_shp = 'Orenburg_training_samples.shp'
s1_csv = 'Sentinel1.csv'
s2_csv = 'Sentinel2.csv'

# load shapefiles
g_ferg = gpd.read_file(ferg_shp)
g_oren = gpd.read_file(oren_shp)
g_all = pd.concat([g_ferg, g_oren], axis=0).reset_index(drop=True)
# ensure ID and label exist; rename if necessary:
# g_all must contain columns 'ID' and 'label' or similar
labels = g_all[['ID', 'Cropland', 'geometry']].copy()

#Load the sentinel data 
s1 = pd.read_csv(s1_csv, parse_dates=['date'])
s2 = pd.read_csv(s2_csv, parse_dates=['date'])


In [25]:
g_ferg.shape

(500, 3)

## Feature functions

In [7]:
def compute_basic_stats(arr):
    arr = np.array(arr, dtype=float)
    arr = arr[~np.isnan(arr)]
    if arr.size == 0:
        return {
            'min': np.nan, 'max': np.nan, 'mean': np.nan, 'median': np.nan,
            'std': np.nan, 'skew': np.nan, 'kurt': np.nan,
            'q10': np.nan, 'q25': np.an, 'q75': np.nan, 'q90': np.nan, 'n': 0
        }
    return {
        'min': np.min(arr), 'max': np.max(arr), 'mean': np.mean(arr),
        'median': np.median(arr), 'std': np.std(arr), 'skew': skew(arr) if arr.size > 2 else 0,
        'kurt': kurtosis(arr) if arr.size > 3 else 0,
        'q10': np.percentile(arr, 10), 'q25': np.percentile(arr, 25),
        'q75': np.percentile(arr, 75), 'q90': np.percentile(arr, 90), 'n': arr.size
    }

In [8]:
def fourier_coeffs(series, ncoeff=2):
    # fillna with linear interp for FFT only (small windows)
    if len(series) < 3:
        return [np.nan]*(ncoeff*2)
    s = pd.Series(series).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values
    coeffs = np.abs(rfft(s))
    #Keep first ncoeff (Excluding DC)
    return coeffs[1:1+ncoeff].tolist() + coeffs[1+ncoeff:1+2*ncoeff].tolist()


## Aggregate per ID

In [9]:
def make_features_for_id(id_val, s1_df, s2_df):
    """Return a dict of features for single ID"""
    f = {'ID': id_val}
    s1t = s1_df[s1_df['ID']==id_val].sort_values('date')
    s2t = s2_df[s2_df['ID']==id_val].sort_values('date')
    # sentinel2 indices
    if len(s2t):
        # compute NDVI etc per row
        B8 = s2t['B8'].astype(float).values
        B4 = s2t['B4'].astype(float).values
        B3 = s2t['B3'].astype(float).values
        B11 = s2t['B11'].astype(float).values
        ndvi = (B8 - B4) / (B8 + B4 + EPS)
        evi = 2.5 * (B8 - B4) / (B8 + 6*B4 - 7.5*s2t['B2'].astype(float).values + 1 + EPS) if 'B2' in s2t.columns else np.full_like(B8, np.nan)
        ndwi = (B3 - B8) / (B3 + B8 + EPS)
        swirnir = (B11 / (B8 + EPS))
        # add stats
        for name, arr in [('ndvi', ndvi), ('evi', evi), ('ndwi', ndwi), ('swirnir', swirnir)]:
            stats = compute_basic_stats(arr)
            for k,v in stats.items():
                f[f's2_{name}_{k}'] = v
            # other features
            f[f's2_{name}_auc'] = np.trapz(np.nan_to_num(arr), dx=1) if np.any(~np.isnan(arr)) else np.nan
            # date of max
            try:
                idx_max = np.nanargmax(arr)
                f[f's2_{name}_doy_max'] = s2t['date'].iloc[idx_max].dayofyear
            except:
                f[f's2_{name}_doy_max'] = np.nan
            # fourier 1st two coeffs
            fc = fourier_coeffs(arr, ncoeff=2)
            for i, val in enumerate(fc):
                f[f's2_{name}_fft{i+1}'] = val
        f['s2_n_obs'] = len(s2t)
    else:
        # fill s2 features as nan
        for col in ['ndvi','evi','ndwi','swirnir']:
            for k in ['min','max','mean','median','std','skew','kurt','q10','q25','q75','q90','n','auc','doy_max','fft1','fft2']:
                f[f's2_{col}_{k}'] = np.nan
        f['s2_n_obs'] = 0
    # sentinel1 stats
    if len(s1t):
        VH = s1t['VH'].astype(float).values
        VV = s1t['VV'].astype(float).values
        ratio = VH / (VV + EPS)
        for name, arr in [('VH', VH), ('VV', VV), ('VH_VV_ratio', ratio)]:
            stats = compute_basic_stats(arr)
            for k,v in stats.items():
                f[f's1_{name}_{k}'] = v
        f['s1_n_obs'] = len(s1t)
    else:
        for name in ['VH','VV','VH_VV_ratio']:
            for k in ['min','max','mean','median','std','skew','kurt','q10','q25','q75','q90','n']:
                f[f's1_{name}_{k}'] = np.nan
        f['s1_n_obs'] = 0
    # coordinates (if exist)
    # pick first non-null translated lat/lon
    try:
        lat = s2t['translated_lat'].dropna().iloc[0] if len(s2t)>0 else (s1t['translated_lat'].dropna().iloc[0] if len(s1t)>0 else np.nan)
        lon = s2t['translated_lon'].dropna().iloc[0] if len(s2t)>0 else (s1t['translated_lon'].dropna().iloc[0] if len(s1t)>0 else np.nan)
    except:
        lat, lon = np.nan, np.nan
    f['lat'] = lat; f['lon'] = lon
    return f

In [ ]:
ids = sorted(set(s1['ID'].unique()).union(set(s2['ID'].unique())))
rows = []
for i, idv in tqdm(enumerate(ids)):
    rows.append(make_features_for_id(idv, s1, s2))
    if i%500==0:
        print("Processed", i)
df_feat = pd.DataFrame(rows)

C:\Users\Reinhard\AppData\Local\Temp\ipykernel_6596\1352690454.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s = pd.Series(series).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values


Processed 0


C:\Users\Reinhard\AppData\Local\Temp\ipykernel_6596\1352690454.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s = pd.Series(series).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values
C:\Users\Reinhard\AppData\Local\Temp\ipykernel_6596\1352690454.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s = pd.Series(series).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values
C:\Users\Reinhard\AppData\Local\Temp\ipykernel_6596\1352690454.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s = pd.Series(series).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values
C:\Users\Reinhard\AppData\Local\Temp\ipykernel_6596\135

Processed 500


C:\Users\Reinhard\AppData\Local\Temp\ipykernel_6596\1352690454.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s = pd.Series(series).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values
C:\Users\Reinhard\AppData\Local\Temp\ipykernel_6596\1352690454.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s = pd.Series(series).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values
C:\Users\Reinhard\AppData\Local\Temp\ipykernel_6596\1352690454.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s = pd.Series(series).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values
C:\Users\Reinhard\AppData\Local\Temp\ipykernel_6596\135

In [31]:
df_feat.head()

,ID,s2_ndvi_min,s2_ndvi_max,s2_ndvi_mean,s2_ndvi_median,s2_ndvi_std,s2_ndvi_skew,s2_ndvi_kurt,s2_ndvi_q10,s2_ndvi_q25,...,s1_VH_VV_ratio_skew,s1_VH_VV_ratio_kurt,s1_VH_VV_ratio_q10,s1_VH_VV_ratio_q25,s1_VH_VV_ratio_q75,s1_VH_VV_ratio_q90,s1_VH_VV_ratio_n,s1_n_obs,lat,lon
0,ID_ABQOQT,-0.058366,0.498492,0.201466,0.197288,0.099195,-0.477076,0.334145,0.078843,0.154497,...,22.918043,731.149919,1.425695,1.661717,2.497947,3.146873,3380,3380,41.510937,72.849414
1,ID_ADDROF,-0.105918,0.671760,0.163903,0.137316,0.116282,1.021568,0.764754,0.040319,0.074190,...,2.286624,738.515763,1.415522,1.655777,2.752278,3.996177,6678,6678,41.380703,71.910726
2,ID_AFIWZH,0.006323,0.871849,0.275762,0.245352,0.165683,0.710096,-0.101521,0.080489,0.149240,...,8.600608,731.081533,1.302201,1.598256,2.638653,4.289795,6686,6686,40.770115,72.190794
3,ID_AFQOFP,-0.037267,0.622854,0.156882,0.122876,0.114539,1.535069,2.357845,0.066482,0.085188,...,2.038824,9.049090,1.245037,1.403532,1.901652,2.296775,6835,6835,41.652317,72.144369
4,ID_AHRONV,-0.096360,0.363734,0.153252,0.157676,0.101214,-0.264042,-0.648274,-0.007771,0.103272,...,1.221908,2.117201,1.293001,1.463740,1.973298,2.330657,751,751,51.642079,54.717496


In [17]:
labels.head()

,ID,Cropland,geometry
0,1,0,POINT (71.74908 41.39632)
1,2,0,POINT (72.90143 41.07219)
2,3,1,POINT (72.46399 40.61906)
3,4,0,POINT (72.19732 41.62759)
4,5,0,POINT (71.31204 41.36946)


In [34]:
# merge labels
#df = df_feat.merge(labels[['ID','Cropland']], on='ID', how='left')
#df = df.dropna(subset=['Cropland'])  # if some are unlabeled


## SECOND SOLUTION

In [35]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb

In [36]:
fergana_gdf = gpd.read_file('Fergana_training_samples.shp')
orenburg_gdf = gpd.read_file('Orenburg_training_samples.shp')

labels_gdf = pd.concat([fergana_gdf, orenburg_gdf], ignore_index = True)
labels_gdf.head()

,ID,Cropland,geometry
0,1,0,POINT (71.74908 41.39632)
1,2,0,POINT (72.90143 41.07219)
2,3,1,POINT (72.46399 40.61906)
3,4,0,POINT (72.19732 41.62759)
4,5,0,POINT (71.31204 41.36946)


In [37]:
sent1 = pd.read_csv('Sentinel1.csv')
sent2 = pd.read_csv('Sentinel2.csv')

sent1.head()

,ID,VH,VV,date,orbit,polarization,rel_orbit,translated_lat,translated_lon
0,ID_AFQOFP,-21.479683,-16.633259,2022-07-01,DESCENDING,"[VV, VH]",78.0,41.652292,72.144256
1,ID_AFQOFP,-24.769110,-15.943674,2022-07-01,DESCENDING,"[VV, VH]",78.0,41.652289,72.144375
2,ID_AFQOFP,-25.370838,-15.185609,2022-07-01,DESCENDING,"[VV, VH]",78.0,41.652286,72.144495
3,ID_AFQOFP,-24.134005,-16.351102,2022-07-01,DESCENDING,"[VV, VH]",78.0,41.652283,72.144614
4,ID_AFQOFP,-20.654249,-16.792723,2022-07-01,DESCENDING,"[VV, VH]",78.0,41.652280,72.144733


In [ ]:
sent1_gdf = gpd.GeoDataFrame(sent1, geometry=[Point(xy) for xy in zip(sent1.translated_lon, sent1.translated_lat)], crs='EPSG:4326')
sent2_gdf = gpd.GeoDataFrame(sent2, geometry=[Point(xy) for xy in zip(sent2.translated_lon, sent2.translated_lat)], crs='EPSG:4326')


In [ ]:
# Spatial join with shapefiel labels
sent1_labeled = gpd.sjoin(sent1_gdf, labels_gdf, how='inner', predicate='nearest')
sent2_labeled = gpd.sjoin(sent2_gdf, labels_gdf, how='inner', predicate='nearest')

sent1_labeled.head()

In [ ]:
#Aggregate features by ID
#Setinel1 take mean values of VH, VV etc. over time
sent1_agg = sent1_labeled

In [ ]:
# REQUIRED PACKAGES
# pip install pandas numpy geopandas scikit-learn lightgbm catboost xgboost shap scipy

import os, gc
import pandas as pd
import numpy as np
import geopandas as gpd
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from scipy.stats import skew, kurtosis
from scipy.fft import rfft
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

EPS = 1e-9

#######################
# 1. Read labels (shapefile) and sentinel CSVs
#######################
# paths
ferg_shp = "Fergana_training_samples.shp"
oren_shp = "Orenburg_training_samples.shp"
s1_csv = "sentinel1.csv"
s2_csv = "sentinel2.csv"

# load shapefiles
g_ferg = gpd.read_file(ferg_shp)
g_oren = gpd.read_file(oren_shp)
g_all = pd.concat([g_ferg, g_oren], axis=0).reset_index(drop=True)
# ensure ID and label exist; rename if necessary:
# g_all must contain columns 'ID' and 'label' or similar
labels = g_all[['ID', 'label', 'geometry']].copy()

# load sentinel data
s1 = pd.read_csv(s1_csv, parse_dates=['date'])
s2 = pd.read_csv(s2_csv, parse_dates=['date'])

# If sentinel CSVs don't have ID aligning, spatial join:
# (example) create a KD-tree to map points -> nearest label ID (only if necessary)
# ----- skip here: assume ID matches -----

#######################
# 2. Feature functions
#######################
def compute_basic_stats(arr):
    arr = np.array(arr, dtype=float)
    arr = arr[~np.isnan(arr)]
    if arr.size == 0:
        return {
            'min': np.nan, 'max': np.nan, 'mean': np.nan, 'median': np.nan,
            'std': np.nan, 'skew': np.nan, 'kurt': np.nan,
            'q10': np.nan, 'q25': np.nan, 'q75': np.nan, 'q90': np.nan, 'n': 0
        }
    return {
        'min': np.min(arr),'max': np.max(arr),'mean': np.mean(arr),
        'median': np.median(arr),'std': np.std(arr),'skew': skew(arr) if arr.size>2 else 0,
        'kurt': kurtosis(arr) if arr.size>3 else 0,
        'q10': np.percentile(arr, 10),'q25': np.percentile(arr, 25),
        'q75': np.percentile(arr, 75),'q90': np.percentile(arr, 90),'n': arr.size
    }

def fourier_coeffs(series, ncoeff=2):
    # fillna with linear interp for FFT only (small windows)
    if len(series) < 3:
        return [np.nan]*(ncoeff*2)
    s = pd.Series(series).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values
    coeffs = np.abs(rfft(s))
    # keep first ncoeff (excluding DC)
    return coeffs[1:1+ncoeff].tolist() + coeffs[1+ncoeff:1+2*ncoeff].tolist()

#######################
# 3. Aggregate per ID
#######################
def make_features_for_id(id_val, s1_df, s2_df):
    """Return a dict of features for single ID"""
    f = {'ID': id_val}
    s1t = s1_df[s1_df['ID']==id_val].sort_values('date')
    s2t = s2_df[s2_df['ID']==id_val].sort_values('date')

    # sentinel2 indices
    if len(s2t):
        # compute NDVI etc per row
        B8 = s2t['B8'].astype(float).values
        B4 = s2t['B4'].astype(float).values
        B3 = s2t['B3'].astype(float).values
        B11 = s2t['B11'].astype(float).values
        ndvi = (B8 - B4) / (B8 + B4 + EPS)
        evi = 2.5 * (B8 - B4) / (B8 + 6*B4 - 7.5*s2t['B2'].astype(float).values + 1 + EPS) if 'B2' in s2t.columns else np.full_like(B8, np.nan)
        ndwi = (B3 - B8) / (B3 + B8 + EPS)
        swirnir = (B11 / (B8 + EPS))
        # add stats
        for name, arr in [('ndvi', ndvi), ('evi', evi), ('ndwi', ndwi), ('swirnir', swirnir)]:
            stats = compute_basic_stats(arr)
            for k,v in stats.items():
                f[f's2_{name}_{k}'] = v
            # other features
            f[f's2_{name}_auc'] = np.trapz(np.nan_to_num(arr), dx=1) if np.any(~np.isnan(arr)) else np.nan
            # date of max
            try:
                idx_max = np.nanargmax(arr)
                f[f's2_{name}_doy_max'] = s2t['date'].iloc[idx_max].dayofyear
            except:
                f[f's2_{name}_doy_max'] = np.nan
            # fourier 1st two coeffs
            fc = fourier_coeffs(arr, ncoeff=2)
            for i, val in enumerate(fc):
                f[f's2_{name}_fft{i+1}'] = val
        f['s2_n_obs'] = len(s2t)
    else:
        # fill s2 features as nan
        for col in ['ndvi','evi','ndwi','swirnir']:
            for k in ['min','max','mean','median','std','skew','kurt','q10','q25','q75','q90','n','auc','doy_max','fft1','fft2']:
                f[f's2_{col}_{k}'] = np.nan
        f['s2_n_obs'] = 0

    # sentinel1 stats
    if len(s1t):
        VH = s1t['VH'].astype(float).values
        VV = s1t['VV'].astype(float).values
        ratio = VH / (VV + EPS)
        for name, arr in [('VH', VH), ('VV', VV), ('VH_VV_ratio', ratio)]:
            stats = compute_basic_stats(arr)
            for k,v in stats.items():
                f[f's1_{name}_{k}'] = v
        f['s1_n_obs'] = len(s1t)
    else:
        for name in ['VH','VV','VH_VV_ratio']:
            for k in ['min','max','mean','median','std','skew','kurt','q10','q25','q75','q90','n']:
                f[f's1_{name}_{k}'] = np.nan
        f['s1_n_obs'] = 0

    # coordinates (if exist)
    # pick first non-null translated lat/lon
    try:
        lat = s2t['translated_lat'].dropna().iloc[0] if len(s2t)>0 else (s1t['translated_lat'].dropna().iloc[0] if len(s1t)>0 else np.nan)
        lon = s2t['translated_lon'].dropna().iloc[0] if len(s2t)>0 else (s1t['translated_lon'].dropna().iloc[0] if len(s1t)>0 else np.nan)
    except:
        lat, lon = np.nan, np.nan
    f['lat'] = lat; f['lon'] = lon

    return f

# Build dataset (example: for speed you should vectorize or parallelize)
ids = sorted(set(s1['ID'].unique()).union(set(s2['ID'].unique())))
rows = []
for i, idv in enumerate(ids):
    rows.append(make_features_for_id(idv, s1, s2))
    if i%500==0:
        print("Processed", i)
df_feat = pd.DataFrame(rows)

# merge labels
df = df_feat.merge(labels[['ID','label']], on='ID', how='left')
df = df.dropna(subset=['label'])  # if some are unlabeled

#######################
# 4. Train LightGBM with spatial group CV
#######################
# create spatial clusters
from sklearn.cluster import MiniBatchKMeans
coords = df[['lat','lon']].fillna(0).values
n_clusters = 20  # tune
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42).fit(coords)
df['spatial_group'] = kmeans.labels_

features = [c for c in df.columns if c not in ['ID','label','spatial_group']]
X = df[features]; y = df['label'].astype(int)

# simple impute: fillna with -999 (trees handle)
X = X.fillna(-999)

oof_preds = np.zeros(len(df))
test_preds = None  # if you have test features

folds = GroupKFold(n_splits=5)
for fold, (tr_idx, val_idx) in enumerate(folds.split(X, y, groups=df['spatial_group'])):
    print("Fold", fold)
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    lgb_train = lgb.Dataset(X_tr, label=y_tr)
    lgb_val = lgb.Dataset(X_val, label=y_val, reference=lgb_train)

    params = {
        'objective':'binary','metric':'binary_logloss',
        'learning_rate':0.03,'num_leaves':31,'max_depth':-1,
        'subsample':0.7,'colsample_bytree':0.6,'reg_alpha':0.5,'reg_lambda':0.8,
        'n_jobs':-1, 'verbose':-1
    }
    clf = lgb.train(params, lgb_train, num_boost_round=5000, valid_sets=[lgb_train, lgb_val],
                    early_stopping_rounds=200, verbose_eval=200)
    oof_preds[val_idx] = clf.predict(X_val, num_iteration=clf.best_iteration)
    # save model if desired

print("OOF Acc:", accuracy_score(y, (oof_preds>0.5).astype(int)))

# Optionally stack OOF preds from multiple base models then train a small LR
